<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/LearningWithoutForgettingno_slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# upload work files from your git hub repository
import sys
!rm -r IncrementalLearning
!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 274, done.
remote: Total 274 (delta 0), reused 0 (delta 0), pack-reused 274
Receiving objects: 100% (274/274), 144.75 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [28]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
import pickle
from torch.autograd import Variable
import pandas as pd
from tqdm.notebook import tqdm
import torch.nn.functional as F

# project classes --------------------------------------------------------------
from IncrementalLearning.cifar100 import ilCIFAR100
from resnet_cifar import resnet32

In [31]:
class lwf(nn.Module):
  def __init__(self,network):
    super(lwf, self).__init__()
    self.model=network.to('cuda')
    self.batch_size=128
    self.CLASSES_PER_BATCH=10
    self.NUM_BATCHES=10
    self.LR=2
    self.MOMENTUM=0.9
    self.WEIGHT_DECAY=1e-5
    self.MILESTONES=[49,63]
    self.GAMMA=0.2
    self.numepochs=70
    self.randomseed=981
    self.trainloader=None
    self.testloader=None
    self.original_training_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed)
    self.original_test_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed, train=False)
    self.criterion=nn.BCEWithLogitsLoss()
   
    
    self.firstbatch=1
    self.classes_seen=0
    self.diz = self.original_training_set.get_dict()
  def forward(self, x):
    self.model = self.model.cuda()
    return self.model.forward(x)
  def update_parameters(self):
    old_model = copy.deepcopy(self)
    old_model.eval()
    old_model.to('cuda')
    n_classes=self.classes_seen+self.CLASSES_PER_BATCH
    print(n_classes)
    optimizer = optim.SGD(self.model.parameters(), lr=self.LR, momentum=self.MOMENTUM, weight_decay=self.WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.MILESTONES, gamma=self.GAMMA)
    for epoch in tqdm(range(self.numepochs)):
      #print(epoch)
        
      for inputs,labels in self.trainloader:
        inputs = Variable(torch.FloatTensor(inputs)).cuda()
        labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])

        labels=Variable(labels).to('cuda')
        optimizer.zero_grad()
        outputs=self.forward(inputs)

        labels_encoded = F.one_hot(labels, 100).float().cuda() #CAMBIARE ONE_HOT
        
        if self.classes_seen:
          old_target = old_model.forward(inputs).cuda()
          old_target = torch.sigmoid(old_target).cuda()
          
          target = torch.cat((old_target[:,:self.classes_seen], labels_encoded[:, self.classes_seen:]), dim=1)
          loss = self.criterion(outputs, target)
        else:
          loss=self.criterion(outputs,labels_encoded) 

        loss.backward()
        optimizer.step()
      
      scheduler.step()

        
              # deep copy the model

  def classify(self, image):
        _, preds = torch.max(torch.softmax(self.forward(image), dim=1), dim=1, keepdim=False)
        return preds

  def __accuracy_on(self, dl, model, mapper):
        total = 0.0
        correct = 0.0
        for  images, labels in dl:
            labels = torch.tensor([torch.tensor(mapper[c.item()]) for c in labels])
            images = Variable(images).cuda()
            preds = model.classify(images)
            total = total + len(labels)
            correct += (preds.data.cpu() == labels).sum()

        acc = 100 * correct / total
        return acc
  
  def training_model(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    current_test_indexes=[]
    
    
    acc=[]
    accuracy=0
    for i in range(self.NUM_BATCHES):
      print(train_indices[i])
      train_dataset=Subset(self.original_training_set,train_indices[i])
      current_test_indexes+=test_indices[i].tolist()
      test_dataset=Subset(self.original_test_set,current_test_indexes)
      self.trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)
      self.testloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)        
      self.train()
      self.update_parameters()
      print('si porco dio')      
      self.classes_seen+=10
      self.eval() # Set Network to evaluation mode
      print('accuracy on training set:', self.__accuracy_on(self.trainloader,self,self.diz))
      print('accuracy on test set:', self.__accuracy_on(self.testloader,self,self.diz))
      print('-' * 80)
      acc.append(accuracy)
    return self


In [32]:
seed = 981
model=lwf(network=resnet32(num_classes=100)).cuda()
model.training_model()

Files already downloaded and verified
Files already downloaded and verified
[    7    13    21 ... 49984 49996 49998]
10



si porco dio
accuracy on training set: tensor(99.5994)
accuracy on test set: tensor(86.2723)
--------------------------------------------------------------------------------
[    0     6    10 ... 49970 49974 49986]
20



si porco dio
accuracy on training set: tensor(95.9736)
accuracy on test set: tensor(70.2083)
--------------------------------------------------------------------------------
[    4    11    16 ... 49972 49976 49980]
30



si porco dio
accuracy on training set: tensor(94.8718)
accuracy on test set: tensor(58.4918)
--------------------------------------------------------------------------------
[    1     3     5 ... 49981 49990 49995]
40



si porco dio
accuracy on training set: tensor(92.5481)
accuracy on test set: tensor(50.3780)
--------------------------------------------------------------------------------
[   12    27    50 ... 49968 49991 49999]
50


...


si porco dio
accuracy on training set: tensor(89.8438)
accuracy on test set: tensor(34.4907)
--------------------------------------------------------------------------------
[   17    26    29 ... 49987 49989 49994]
80



si porco dio
accuracy on training set: tensor(91.4263)
accuracy on test set: tensor(31.0988)
--------------------------------------------------------------------------------
[    9    22    25 ... 49914 49985 49992]
90



si porco dio
accuracy on training set: tensor(88.9223)
accuracy on test set: tensor(28.1920)
--------------------------------------------------------------------------------
[    2    20    34 ... 49929 49942 49993]
100



si porco dio
accuracy on training set: tensor(87.2796)
accuracy on test set: tensor(24.3990)
--------------------------------------------------------------------------------


lwf(
  (model): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=